In [ ]:
# katib 설치 확인
!pip freeze | grep katib

In [ ]:
# 가지고 있는 expermient 확인
from kubeflow import katib
from kubeflow.katib import KatibClient

kclient = KatibClient()
kclient.list_experiments()

In [ ]:
from kubernetes.client import V1ObjectMeta
from kubeflow.katib import V1beta1Experiment
from kubeflow.katib import V1beta1AlgorithmSpec
from kubeflow.katib import V1beta1ObjectiveSpec
from kubeflow.katib import V1beta1FeasibleSpace
from kubeflow.katib import V1beta1ExperimentSpec
from kubeflow.katib import V1beta1ObjectiveSpec
from kubeflow.katib import V1beta1ParameterSpec
from kubeflow.katib import V1beta1TrialTemplate
from kubeflow.katib import V1beta1TrialParameterSpec

experiment_name = "fashion-mnist-tuner-sdk2"

metadata = V1ObjectMeta(
    name=experiment_name
)


algorithm_spec=V1beta1AlgorithmSpec(
    algorithm_name="random"
)

objective_spec=V1beta1ObjectiveSpec(
    type="maximize",
    goal= 0.99,
    objective_metric_name="Validation-accuracy",
    additional_metric_names=["Validation-loss", "accuracy", "loss"]
)

parameters=[
    V1beta1ParameterSpec(
        name="node_amount",
        parameter_type="categorical",
        feasible_space=V1beta1FeasibleSpace(
            list=["64", "128", "256", "521"],
        ),
    ),
    V1beta1ParameterSpec(
        name="dropout_rate",
        parameter_type="double",
        feasible_space=V1beta1FeasibleSpace(
            min="0.1",
            max="0.9"
        ),
    ),
    V1beta1ParameterSpec(
        name="optimizer",
        parameter_type="categorical",
        feasible_space=V1beta1FeasibleSpace(
            list=["sgd", "adam", "nadam", "rmsprop"],
        ),
    ),  
    V1beta1ParameterSpec(
        name="epoch",
        parameter_type="int",
        feasible_space=V1beta1FeasibleSpace(
           min="10",
           max="30"
        ),
    ),  
]

# JSON template specification for the Trial's Worker Kubernetes Job.
trial_spec = {
    "apiVersion": "batch/v1",
    "kind": "Job",
    "spec": {
        "template": {
            "metadata": {
                "annotations": {
                    "sidecar.istio.io/inject": "false"
                }
            },
            "spec": {
                "containers": [
                    {
                        "name": "training-container",
                        "image": "yourID/fashion-mnist-tuner:yourTAG", 
                        "command": [
                            "python",
                            "/app/fashion_mnist_tuner.py",
                            "--node_amount=${trialParameters.node_amount}",
                            "--dropout_rate=${trialParameters.dropout_rate}",
                            "--optimizer=${trialParameters.optimizer}",
                            "--epoch=${trialParameters.epoch}"
                        ]
                    }
                ],
                "restartPolicy": "Never"
            }
        }
    }
}


# Configure parameters for the Trial template.
trial_template=V1beta1TrialTemplate(
    primary_container_name="training-container",
    trial_spec=trial_spec,    
    trial_parameters=[
        V1beta1TrialParameterSpec(
            name="node_amount",
            description="Layer neuron amount",
            reference="node_amount"
        ),        
        V1beta1TrialParameterSpec(
            name="dropout_rate",
            description="dropout_rate for the training model",
            reference="dropout_rate"
        ),
        V1beta1TrialParameterSpec(
            name="optimizer",
            description="optimizer for the training model",
            reference="optimizer"
        ),
        V1beta1TrialParameterSpec(
            name="epoch",
            description="epoch for the training model",
            reference="epoch"
        )
    ]
)


# Experiment object.
experiment = V1beta1Experiment(
    api_version="kubeflow.org/v1beta1",
    kind="Experiment",
    metadata=metadata,
    spec=V1beta1ExperimentSpec(
        max_trial_count=12,
        parallel_trial_count=3,
        max_failed_trial_count=3,
        algorithm=algorithm_spec,
        objective=objective_spec,
        parameters=parameters,
        trial_template=trial_template,
    )
)


In [ ]:
# sdk로 experiment 생성하기
kclient.create_experiment(experiment)

In [ ]:
# tuner의 탐색 상태 (experiment) 확인하기
kclient.get_experiment_status("fashion-mnist-tuner-sdk")